In [63]:
import pandas as pd
import os
from os import path
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score
from imblearn.metrics import classification_report_imbalanced
from sklearn import svm
import matplotlib.pyplot as plt

base_path = '/data/pixel-annot-export-v0.9/'
ann_path = path.join(base_path, 'pixel_df_list/')

# TMP DIRECTORY PATH FOR TMP GENSIM OUTPUT
opref = '/data/katya/tmp/ionsim'

ds_df_path = path.join(base_path, 'ds_df.msgpack')
ds_df = pd.read_msgpack(ds_df_path)

ion_df_path = path.join(base_path, 'ion_df.msgpack')
ion_df = pd.read_msgpack(ion_df_path)

formula_df_path = path.join(base_path, 'formula_df.msgpack')
formula_df = pd.read_msgpack(formula_df_path)

gs_path = '/data/katya/offsample_gs/GS.csv'
gs_df = pd.read_csv(gs_path)

# PATH TO ION SIMILARITIES GENERATED BY generate_colloc_sim_vectors.ipynb
sim_features_dir = '/data/katya/offsample_gs_vectors_norm'

In [54]:
# Get ion indices from dataframe
def get_ion_ind_df(ds_rows):
    result = []
    for _, row in ds_rows.iterrows():
        ion_ind_list = ion_df[ion_df.formula == row.sumFormula][ion_df.adduct == row.adduct].index.tolist()
        if len(ion_ind_list) == 0:
            print('No ion in the dictionary: ', sf, adduct)
        else:
            ion_ind = ion_ind_list[0]
            result.append(ion_ind)
    return result

In [67]:
diff_df = pd.DataFrame()
counter = 0
for f in os.listdir(sim_features_dir):
    print(counter, f)
    counter += 1
    ds_ind = int(f)
    datasetId = ds_df['id'].loc[ds_ind]
    ds_rows = gs_df[gs_df.datasetId == datasetId]
        
    ann_on_ions = get_ion_ind_df(ds_rows[ds_rows.type == 'on'])
    ann_off_ions = get_ion_ind_df(ds_rows[ds_rows.type == 'off'])
        
    if len(ann_on_ions)==0 and len(ann_off_ions)==0: continue
    
    sim_df = pd.read_msgpack(path.join(sim_features_dir, f))
   
    ion_sub_df = ion_df.loc[sim_df.index]
    
    sim_df = sim_df.loc[ann_off_ions+ann_on_ions]
        
    # replace ions with formulas
    for sf, rows in ion_sub_df.groupby('formula'):
        ion_columns = rows.index.tolist()
        sim_df[sf] = sim_df[ion_columns].max(axis=1)
        sim_df = sim_df.drop(ion_columns, axis=1)
                      
    label_df = pd.DataFrame([(ds_ind, 1) for i in range(0,len(ann_off_ions))], columns = ['ds_ind', 'class'], index = ann_off_ions)
    label_df = label_df.append(pd.DataFrame([(ds_ind, 0) for i in range(0,len(ann_on_ions))], columns = ['ds_ind', 'class'], index = ann_on_ions))

    sim_df = sim_df.merge(label_df, left_index=True, right_index=True)
    diff_df = diff_df.append(sim_df)

0 1773


/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


1 511
2 94
3 497
4 692
5 425
6 1439
7 2163
8 566
9 290
10 2233
11 2336
12 1182
13 888
14 471
15 944
16 510
17 1248
18 677
19 847
20 1097
21 1093
22 2297
23 690
24 1435
25 371
26 129
27 673
28 1581
29 441
30 687
31 2448
32 1240
33 190
34 885
35 697
36 882
37 51
38 75
39 835
40 599
41 1771
42 98
43 2449
44 1395
45 1402
46 1772
47 1394
48 1307
49 1094
50 1942
51 47
52 1804
53 1352
54 1410
55 1176
56 1177
57 322
58 301
59 1105
60 2300
61 1104
62 708
63 1566
64 2104
65 935
66 601
67 1142
68 2330
69 848
70 821
71 2313
72 1090
73 1450
74 2293
75 2291
76 709
77 2024
78 193
79 696
80 555
81 93
82 403
83 2329
84 53
85 672
86 685


In [68]:
print(diff_df.shape)
diff_df.fillna(0, inplace=True)
print(len(diff_df.notnull()))
print(diff_df.columns)

(23238, 10535)
23238
Index(['C10H10ClN3', 'C10H10N2O', 'C10H10N2O2', 'C10H10N2O3S', 'C10H10N2O4S',
       'C10H10N2OS', 'C10H10N4O', 'C10H10N4O2S', 'C10H10N7O2S', 'C10H10O',
       ...
       'CH5AsO2', 'CH5AsO3', 'CH5N3', 'CH5O4P', 'CHBr3', 'CHBrCl2', 'CHCl3',
       'CHF3O3S', 'class', 'ds_ind'],
      dtype='object', length=10535)


In [42]:
# leave only DHB positive
exclude_datasets = []
ds_inds = diff_df.ds_ind.unique()

c = 0
for ds_ind in ds_inds:
    ds_name = ds_df.loc[ds_ind]['name']
    if ds_df.loc[ds_ind]['ionisationSource'] != 'MALDI' or ds_df.loc[ds_ind]['maldiMatrix'] != '2,5-dihydroxybenzoic acid (DHB)':
        exclude_datasets.append(ds_ind)
        print(ds_name)
        continue
    else: c += 1
print(c)

PS_7_P7_N1_30um_New
AstraZeneca//Rat_testis
Dataset 23
MPIMM_049_QE_P_BA_CF_sp3
North Carolina State University//mouse body
DESI porcine kidney interface region
20180220_mouse eye_DHB
Dataset 26_recal
ICL//LNTO42_17_1
Borderline14
ICL//LNTO31_17_4
AstraZeneca//CT26_xenograft
100um_noM2_001_Recal
Dataset 8
ICL//A51 CT S3-centroid
Dataset 21
20170905_CGL0170817_MT-MB_ATP_N_81x101_135x135
170321_kangaroobrain-bpyn1-pos_maxof50.0_med1r
Dataset 4
Borderline13
Dataset 18
Tumor41
ICL//LNTO41_17_4
Servier_Ctrl_rat_liver_9aa
ICL//LNTO41_17_2
xenograft CT-26 S10
Normal8
ICL//LNTO30_16_1
NCSU_M8_mouse_brain
20170815_CGL_MT-M.B_DAN012_NTM_42x30_100x100
Servier_Ctrl_mouse_wb_median_plane_9aa
Xenograft_3_135um
BRB04S-RECAL
Mouse_Wholebody_3
20170817_CGL_MT-M.B_ATP_DAN018_NTM_102x39_100x100
Dataset 3_recal
Dataset 4_recal
Tumor17
NCSU_mouse_LCC6_tumor
Dataset 2
servier_TT_mouse_wb_fmpts_derivatization_CHCA
Dataset 30
ICL//LNTO26_7_3
20170503_ADP-JS_CD1-Brain_dan005_117x143_60x60_N
ICL//7TopRight,_17B

In [44]:
import random 
def partition(list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]
gs_inds = diff_df.ds_ind.unique()
incl_inds = [i for i in gs_inds if i not in exclude_datasets]
folds = partition(incl_inds, 5)
print(folds)

[[2330, 1395, 1097, 1772, 2448, 2293, 1093], [1090, 1240, 2329, 1773, 2163, 1942], [2297, 1450, 1581, 1439, 1394, 2291], [2104, 1094, 566, 1771, 511, 2300], [2313, 1566, 2449, 129, 98, 1142]]


In [69]:
exclude_datasets = []
vitaly_folds = [['DESI porcine kidney interface region',
  '20180220_mouse eye_DHB',
  'Servier_Ctrl_mouse_wb_median_plane_DHB',
  'slide012_animal102_minipig_kidney- total ion count',
  'Dataset 31',
  'xenograft CT-26 S10',
  'Dataset 26_recal',
  '170321_kangaroobrain-bpyn1-pos_maxof50.0_med1r',
  'Servier_Ctrl_mouse_wb_median_plane_9aa',
  'LNTO31_17_1-RECAL',
  '2017023_ADP_GFlarvae_55x55_40x40_DMAN002',
  '161007_WT1S1L1',
  'servier_TT_mouse_wb_fmpts_derivatization_CHCA',
  'ICL//LNTO30_16_1',
  'ICL//LNTO42_17_1',
  'ICL//LNTO41_17_4',
  '50%meoh_8cyc_75um'],
 ['Esco2_centroids',
  '20170503_ADP-JS_CD1-Brain_dan005_117x143_60x60_N',
  'Dataset 23',
  'Dataset 4',
  'Mouse Kidney on plastic slide 260x175 30um',
  '20170815_CGL_MT-M.B_DAN012_NTM_42x30_100x100',
  'S648 WS20 205x170 20um E110',
  'MP 39',
  '50um_min(focusing)_high(m_z)_dry(application)',
  '20170817_CGL_MT-M.B_ATP_DAN018_NTM_102x39_100x100',
  'UoNotreDame//Untreated_6_574',
  'NCSU_mouse_LCC6_tumor',
  'AstraZeneca//CT26_xenograft',
  'MPIMM_049_QE_P_BA_CF_sp3',
  'Dataset 4_recal',
  'Mouse Kidney on coated plastic slide 270x160 30 um',
  'NCSU_M8_mouse_brain',
  '20170905_CGL0170817_MT-MB_ATP_N_81x101_135x135'],
 ['Servier_Ctrl_mouse_wb_lateral_plane_DHB',
  'AstraZeneca//Rat_testis',
  'Dataset 30',
  '100um_noM2_001_Recal',
  'Row001',
  '20171110_94T_RDAM_1b',
  'Servier_Ctrl_rat_liver_9aa',
  'PS_7_P7_N1_30um_New',
  'Dataset 27',
  'BRB04S-RECAL',
  'Dataset 21',
  'ICL//LNTO41_17_2',
  'Esco1_centroids',
  '75um_small(focusing)_low(m_z)_dry(application)',
  'DESI quan_Swales',
  'Xenograft_3_135um',
  'ST-Pseudo_centroids',
  'North Carolina State University//mouse body'],
 ['Dataset 18',
  'ICL//A51 CT S3-centroid',
  'Dataset 16',
  'Borderline14',
  'Rat50_Section1',
  'Tumor17',
  'ICL//LNTO22_1_3',
  'ICL//7TopRight,_17BottomRight,_27BottomLeft,_37TopLeft',
  'MPIMM_021_QE_P_CB_0',
  '20170105_VS_disc2_NEDC0001_70x80_10x10',
  'slide077_animal121_rat_kidney - total ion count',
  'Normal8',
  'ICL//LNTO31_17_4',
  'Dataset 8',
  'Mousebrain_MG08_2017_GruppeA',
  'ICL//LNTO26_7_3',
  'Mouse Kidney on coated plastic slide 300x165 30 um'],
 ['Dataset 3',
  'Mouse_Wholebody_3',
  'Rat51_Section2',
  '12_cylces_75um_new_submission',
  'CC-Esco2_centroids',
  'Dataset 2',
  'Tumor11',
  '70%meoh_8cyc_75um',
  '170418_MouseBrain_pos_50um_205by129_atn0',
  'Borderline13',
  'Kidney_280x140_40um_E0',
  'S649 WS21 155x105 15um E110',
  'MS7_20170907_DHB_pos_MouseBrain_s3_300-1800_40um_245x134_A30-JBE-Test',
  'Tumor41',
  'HK_S2_N2_20um_New',
  '75um_small(focusing)_high(m_z)_dry(application)',
  'Dataset 3_recal']]

folds = []
diff_inds = diff_df.ds_ind.unique()
print(diff_df.ds_ind.unique())
for ds_names in vitaly_folds:
    ds_inds = []
    for dn in ds_names:
        dn = dn.replace('__','//')
        dn = dn.replace('m_z','m/z')
        inds = ds_df[ds_df["name"] == dn].index.tolist()
        if len(inds) > 1: 
            ds_inds.append([i for i in inds if i in diff_inds][0])
        else: ds_inds.append(inds[0])
    if len(ds_names) != len(ds_inds): print('problem!')
    folds.append(ds_inds)

[1773  511   94  497  692  425 1439 2163  566  290 2233 2336 1182  888
  471  944  510 1248  677  847 1097 1093 2297  690 1435  371  129  673
 1581  441  687 2448 1240  190  885  697  882   51   75  835  599 1771
   98 2449 1395 1402 1772 1394 1307 1094 1942   47 1804 1352 1410 1176
 1177  322  301 1105 2300 1104  708 1566 2104  935  601 1142 2330  848
  821 2313 1090 1450 2293 2291  709 2024  193  696  555   93  403 2329
   53  672  685]


In [73]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from imblearn.metrics import classification_report_imbalanced

clf = SVC(kernel="linear", C=0.025)
param = {}

f1_off = []
f1_on = []
p_off = []
p_on = []
r_off = []
r_on = []

classification_results = pd.DataFrame(columns = ['ds_ind', 'OFF-F1', 'OFF-P', 'OFF-R', 'ON-F1', 'ON-P', 'ON-R', '#PREDICTED_OFF'])

for i, batch in enumerate(folds):        
    test_ds = batch
    train_ds = [item for j, sublist in enumerate(folds) if (j not in exclude_datasets) and (j!=i) for item in sublist]
    
    ds_rows = diff_df[diff_df.ds_ind.isin(test_ds)]
    non_ds_rows = diff_df[(diff_df.ds_ind.isin(train_ds))] 
    
    train_target = non_ds_rows.as_matrix(columns = ['class'])
    train_data = non_ds_rows.drop(['class', 'ds_ind'], axis=1).as_matrix()
    c, r = train_target.shape
    train_target = train_target.reshape(c,)
        
    test_target = ds_rows.as_matrix(columns = ['class'])
    test_data = ds_rows.drop(['class', 'ds_ind'], axis=1).as_matrix()
    c, r = test_target.shape
    test_target = test_target.reshape(c,)
    
    print('FOLD', i)
    
    clf.fit(train_data, train_target)
    prediction = clf.predict(test_data)
        
    off_f1 = f1_score(test_target, prediction, pos_label = 1)
    off_p = precision_score(test_target, prediction, pos_label = 1)
    off_r = recall_score(test_target, prediction, pos_label = 1)
    
    f1_off.append(off_f1)
    p_off.append(off_p)
    r_off.append(off_r)
    
    on_f1 = f1_score(test_target, prediction, pos_label = 0)
    on_p = precision_score(test_target, prediction, pos_label = 0)
    on_r = recall_score(test_target, prediction, pos_label = 0)
    
    
    f1_on.append(on_f1)
    p_on.append(on_p)
    r_on.append(on_r)
    
    print(classification_report_imbalanced(test_target, prediction, target_names=['On sample', 'Off sample']))
    
    classification_results.loc[i] = [ds_ind, off_f1, off_p, off_r, on_f1, on_p, on_r, sum(prediction)]
    
print()
    
print("F1 ON average with %d batches: %0.2f (+/- %0.2f)" % (len(f1_on), np.mean(f1_on), np.std(f1_on) * 2))
print("P ON average: %0.2f (+/- %0.2f)" % (np.mean(p_on), np.std(p_on) * 2))
print("R ON average: %0.2f (+/- %0.2f)" % (np.mean(r_on), np.std(r_on) * 2))

f1_on_nozero = [v for v in f1_on if v>0.0]
p_on_nozero = [v for v in p_on if v>0.0]
r_on_nozero = [v for v in r_on if v>0.0]

print("F1 ON nonzero with size %d: %0.2f (+/- %0.2f)" % (len(f1_on_nozero), np.mean(f1_on_nozero), np.std(f1_on_nozero) * 2))
print("P ON nonzero with size %d: %0.2f (+/- %0.2f)" % (len(p_on_nozero), np.mean(p_on_nozero), np.std(p_on_nozero) * 2))
print("R ON nonzero with size %d: %0.2f (+/- %0.2f)" % (len(r_on_nozero), np.mean(r_on_nozero), np.std(r_on_nozero) * 2))

print()

print("F1 OFF with %d batches: %0.2f (+/- %0.2f)" % (len(f1_off), np.mean(f1_off), np.std(f1_off) * 2))
print("P OFF average: %0.2f (+/- %0.2f)" % (np.mean(p_off), np.std(p_off) * 2))
print("R OF average: %0.2f (+/- %0.2f)" % (np.mean(r_off), np.std(r_off) * 2))

f1_off_nozero = [v for v in f1_off if v>0.0]
p_off_nozero = [v for v in p_off if v>0.0]
r_off_nozero = [v for v in r_off if v>0.0]

print("F1 OFF nonzero with size %d: %0.2f (+/- %0.2f)" % (len(f1_off_nozero), np.mean(f1_off_nozero), np.std(f1_on_nozero) * 2))
print("P OFF nonzero with size %d: %0.2f (+/- %0.2f)" % (len(p_off_nozero), np.mean(p_off_nozero), np.std(p_off_nozero) * 2))
print("R OFF nonzero with size %d: %0.2f (+/- %0.2f)" % (len(r_off_nozero), np.mean(r_off_nozero), np.std(r_off_nozero) * 2))

FOLD 0
                   pre       rec       spe        f1       geo       iba       sup

  On sample       0.93      0.99      0.84      0.96      0.91      0.84      3160
 Off sample       0.97      0.84      0.99      0.90      0.91      0.81      1480

avg / total       0.94      0.94      0.88      0.94      0.91      0.83      4640

FOLD 1
                   pre       rec       spe        f1       geo       iba       sup

  On sample       0.93      0.94      0.94      0.94      0.94      0.88      2129
 Off sample       0.95      0.94      0.94      0.95      0.94      0.88      2518

avg / total       0.94      0.94      0.94      0.94      0.94      0.88      4647

FOLD 2
                   pre       rec       spe        f1       geo       iba       sup

  On sample       0.80      0.98      0.75      0.88      0.86      0.75      2402
 Off sample       0.97      0.75      0.98      0.84      0.86      0.72      2264

avg / total       0.89      0.87      0.86      0.86      